In [1]:
import numpy as np
from gauss_hf import *

In [2]:
testp = '../tdcis_data/cis-30roots_s0_heh+_sto-3g.log'

In [3]:
datafile = log_data(testp)


Reading data from ../tdcis_data/cis-30roots_s0_heh+_sto-3g.log...
	Gaussian .LOG data read.



In [4]:
dipX = datafile.get_dipole_x_AO()
dipY = datafile.get_dipole_y_AO()
dipZ = datafile.get_dipole_z_AO()

In [5]:
def prefixCIS(mol, basis):
    # check if molecule is represented
    if mol=='heh+' or mol=='h2':
        if basis=='sto-3g' or basis=='6-31g':
            prefix = 'cis-30roots_s0_'
    elif mol=='lih' and basis=='sto-3g':
        prefix = 'cis-30roots_s0_'
    else:
        print("Error: molecular system not recognized!")
        return 1
    return prefix


In [6]:
def get_MOs_CIS(mol, basis, path='../tdcis_data/'):
    ident = prefixCIS(mol,basis)+mol+'_'+basis
    f = open(path+ident+'.log','r')
    extract = False
    NMO = 0
    for x in f:
        if "Eigenvalues and eigenvectors of Harris matrix:" in x:
            extract = True
            continue
        if extract:
            strlist = x.split()
            if len(strlist) > 0:
                if strlist[0] == 'Eigenvalues':
                    NMO = len(strlist) - 2
                    MO = np.zeros((NMO, NMO))
                    MOcount = 0
                    continue
                elif strlist[0] == 'Harris':
                    break
            if NMO > 0:
                for j in range(NMO):
                    MO[MOcount, j] = float(strlist[1 + j])
                MOcount += 1

    return MO

In [7]:
def get_S0den_CIS(mol, basis, path='../tdcis_data/'):
    ident = prefixCIS(mol,basis)+mol+'_'+basis
    f = open(path+ident+'.log','r')
    extract = False
    NMO = 0
    MOcount = 0
    for x in f:
        if "Final density matrix:" in x:
            extract = True
            continue
        if extract:
            strlist = x.split()
            NMO = len(strlist)
            S0den = np.zeros((NMO, NMO))
            extract = False
            continue
        if NMO > 0 and MOcount < NMO:
            strlist = x.split()
            for j in range(MOcount+1):
                S0den[MOcount, j] = np.float64(strlist[1 + j].replace("D","e"))
                if MOcount != j:
                    S0den[j, MOcount] = S0den[MOcount, j]
            MOcount += 1

    return S0den

In [8]:
def get_Sjden_CIS(mol, basis, j, path='../tdcis_data/'):
    ident = prefixCIS(mol,basis)+mol+'_'+basis
    f = open(path+ident+'.log','r')
    extract = False
    skip = False
    startden = False
    NMO = 0
    MOcount = 0
    for x in f:
        if "Excited State   " + str(j) in x:
            extract = True
            continue
        if extract and "CIS Density (Total):" in x:
            skip = True
            continue
        if extract and skip:
            strlist = x.split()
            NMO = len(strlist)
            Sjden = np.zeros((NMO, NMO))
            extract = False
            startden = True
            continue
        if startden and MOcount < NMO:
            strlist = x.split()
            for j in range(MOcount+1):
                Sjden[MOcount, j] = np.float64(strlist[1 + j].replace("D","e"))
                if MOcount != j:
                    Sjden[j, MOcount] = Sjden[MOcount, j]
            MOcount += 1

    return Sjden

In [9]:
# get transition densities from state 0 to state j
# here state 1 means first excited state, etc...
# numexcited = (total number of CI states) - 1
#            = total number of CI excited states
def get_trans0dens_CIS(mol, basis, numexcited, path='../tdcis_data/'):
    ident = prefixCIS(mol,basis)+mol+'_'+basis
    f = open(path+ident+'.log','r')
    extract = False
    startden = False
    whichstate = 1
    alldens = []
    for x in f:
        if whichstate > numexcited:
            break
        if "Alpha transition density to state          " + str(whichstate) in x:
            extract = True
            ind = 0
            startden = False
            NMO = 0
            MOcount = 0
            continue
        if "Beta transition density to state          " + str(whichstate) in x:
            extract = True
            ind = 1
            startden = False
            NMO = 0
            MOcount = 0
            continue
        if extract:
            strlist = x.split()
            NMO = len(strlist)
            if ind==0:
                Sjden = np.zeros((NMO, NMO, 2))
            startden = True
            extract = False
            continue
        if startden and MOcount < NMO:
            strlist = x.split()
            for j in range(NMO):
                Sjden[MOcount, j, ind] = np.float64(strlist[1 + j].replace("D","e"))
            MOcount += 1
            if ind==1 and MOcount==NMO:
                alldens.append(np.sum(Sjden,axis=2))
                whichstate += 1

    return alldens

In [10]:
MO = get_MOs_CIS('heh+','sto-3g')
print(MO)

[[ 0.28492  1.15198]
 [ 0.81735 -0.86034]]


In [11]:
S0den = get_S0den_CIS('heh+','sto-3g')
print(S0den)

[[0.0410154 0.177468 ]
 [0.177468  0.767882 ]]


In [12]:
S1den = get_Sjden_CIS('heh+','sto-3g',1)
print(S1den)

[[ 1.40823  -0.758214]
 [-0.758214  1.40823 ]]


In [19]:
transden = get_trans0dens_CIS('heh+','sto-3g',1)
print(transden)

[array([[ 0.473612,  2.04926 ],
       [-0.324126, -1.402446]])]


In [28]:
transden[0] = np.array([-3.34893882e-01, -1.44904221e+00,  2.29191033e-01,  9.91679749e-01]).reshape((2,2))

In [20]:
# S1den[0,0] = 0.704117
# S1den[1,0] = -0.379107
# S1den[0,1] = -0.379107
# S1den[1,1] = 0.704117

In [21]:
-2*np.trace(S1den @ dipZ)

0.22286821001759982

In [22]:
CIdimatnpz = np.load('./logfiles/casscf22_s2_h2_sto-3g_CI_dimat.npz')

In [23]:
CIdimatnpz

array([[-0.        , -0.        ,  3.48066017, -0.        ],
       [-0.        , -0.        , -0.        , -0.        ],
       [ 3.48066017, -0.        , -0.        ,  0.39423189],
       [-0.        , -0.        ,  0.39423189, -0.        ]])

In [24]:
S = datafile.get_overlap_AO()
print(S)

[[1.       0.538415]
 [0.538415 1.      ]]


In [25]:
MO.T @ S @ MO

array([[ 1.00001191e+00, -1.51789862e-06],
       [-1.51789862e-06,  1.00000257e+00]])

In [31]:
transden[0] * -np.sqrt(2)

array([[ 0.47361147,  2.04925515],
       [-0.32412507, -1.40244695]])

In [29]:
# dipole moment matrix for CIS, in the CIS basis (S0, S1 basis)
# CIS only has two states! (for this system)
np.array([[-2*np.trace(S0den @ dipZ), -1*np.trace(transden[0] @ dipZ)],
          [-1*np.trace(transden[0] @ dipZ), -1*np.trace(S1den @ dipZ)]])

array([[-1.11256708, -0.87800763],
       [-0.87800763,  0.11143411]])

In [192]:
CIdimatnpz

array([[-0.        , -0.        ,  3.48066017, -0.        ],
       [-0.        , -0.        , -0.        , -0.        ],
       [ 3.48066017, -0.        , -0.        ,  0.39423189],
       [-0.        , -0.        ,  0.39423189, -0.        ]])

In [ ]:
# what are we still confused by?
# - sqrt(2) difference between my transition dipole moments and Gaussian's (no CASSCF comparison)
# - CASSCF transition dipole?
# - feel good about S0 dipole
# - not sure about CIS S1 and CASSCF S1, S2 diagonal dipole elements
# - it's weird that S1 doesn't match (also, weirdly, CIS' S1 dipole matches CASSCF T1)

# - electronic moments get printed out --> are they for ground state? 
# - get CIS to print out excited state dipole moment!

# - we don't trust CIS transition density properties (including density itself)
# - we trust CIS eigenvalues & eigenvectors
# - we trust CASSCF

# - use some other codes for h2, heh+
# - use qchem for CIS

In [11]:
0.2573333333 - (-0.5146666667)

0.772

In [12]:
0.772/2

0.386